In [1]:
import model

In [2]:
firstModel = model.IoisModel(verbose = True, improvisation = 10)

In [3]:
%prun firstModel.run_model()

['Time: ', 1, 'Apps: ', 53, 'Pay: ', 2729.318170053808, 'Cost', 3180.0, 'Actor Connections', 577, 'Application Connections', 2832]
['Time: ', 2, 'Apps: ', 54, 'Pay: ', 5985.977496145321, 'Cost', 3359.0, 'Actor Connections', 623, 'Application Connections', 4398]
['Time: ', 3, 'Apps: ', 54, 'Pay: ', 9409.418595381134, 'Cost', 3599.0, 'Actor Connections', 655, 'Application Connections', 5008]
['Time: ', 4, 'Apps: ', 55, 'Pay: ', 12862.46521840202, 'Cost', 4016.0, 'Actor Connections', 708, 'Application Connections', 5604]
['Time: ', 5, 'Apps: ', 55, 'Pay: ', 16242.826694225674, 'Cost', 4496.0, 'Actor Connections', 679, 'Application Connections', 6099]
['Time: ', 6, 'Apps: ', 55, 'Pay: ', 19586.74839239744, 'Cost', 5091.0, 'Actor Connections', 711, 'Application Connections', 6421]
['Time: ', 7, 'Apps: ', 55, 'Pay: ', 23062.412456712787, 'Cost', 5811.0, 'Actor Connections', 763, 'Application Connections', 6563]
['Time: ', 8, 'Apps: ', 55, 'Pay: ', 26477.21565053553, 'Cost', 6651.0, 'Actor Co

         77045796 function calls in 170.596 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     8013   36.606    0.005   93.134    0.012 agents.py:33(communicate)
 11023484   18.550    0.000   22.887    0.000 reportviews.py:783(__iter__)
  8313698   15.540    0.000   46.956    0.000 graph.py:452(__getitem__)
  8313698   12.289    0.000   16.035    0.000 graph.py:338(adj)
  8313698   12.089    0.000   15.381    0.000 coreviews.py:81(__getitem__)
 16627396    7.038    0.000    7.038    0.000 coreviews.py:44(__init__)
   586607    6.986    0.000    6.986    0.000 utility.py:13(match)
      300    5.511    0.018  149.809    0.499 schedule.py:58(step)
  8313698    5.276    0.000    5.276    0.000 coreviews.py:53(__getitem__)
   360000    4.834    0.000    4.834    0.000 {method 'binomial' of 'numpy.random.mtrand.RandomState' objects}
    36378    4.637    0.000   16.890    0.000 agents.py:23(<dictcomp>)
 10977793    4.036    0.

In [6]:
import time
import numpy as np

In [80]:
from mesa import Agent
import numpy as np
from utility import match
from operator import itemgetter

def communicate1(self, schedule):
        """
        The agent communicates
        """
        edgesToAdd = []
        for s,actorKey,d in schedule.network.edges(self.unique_id, data=True):
            if actorKey in schedule.actorList and schedule.actorList[actorKey].tier != self.tier and d['strength'] >= self.model.commThreshold:

                d['strength'] = min(d['strength'] + self.model.commGain, 1)
                
                for app in list(set(schedule.network.neighbors(actorKey)) | set(schedule.network.neighbors(s))):
                    if app not in schedule.actorList:
                        try:
                            ownKnow = schedule.network[self.unique_id][app]['strength']
                        except KeyError:
                            edgesToAdd.append((self.unique_id, app, schedule.network[actorKey][app]['strength'] * self.model.commDensity))
                            continue
                        try:
                            otherKnow = schedule.network[actorKey][app]['strength']
                        except KeyError:
                            edgesToAdd.append((actorKey, app, ownKnow * self.model.commDensity))
                            continue
                        diff = ownKnow - otherKnow
                        if diff > 0:
                            schedule.network[actorKey][app]['strength'] += diff * self.model.commDensity
                        elif diff < 0:
                            schedule.network[self.unique_id][app]['strength'] += -diff * self.model.commDensity
                # as array: (app, node1, node2) aber dann keine Verbindung zu nodes
                
        schedule.network.add_weighted_edges_from(edgesToAdd, 'strength')


def communicate2(self, schedule):
        """
        The agent communicates
        """
        edgesToAdd = []
        for s,actorKey,d in schedule.network.edges(self.unique_id, data=True):
            if actorKey in schedule.actorList and schedule.actorList[actorKey].tier != self.tier and d['strength'] >= self.model.commThreshold:

                d['strength'] = min(d['strength'] + self.model.commGain, 1)
                
                for app in list(set(schedule.network.neighbors(actorKey)) | set(schedule.network.neighbors(s))):
                    if app not in schedule.actorList:
                        try:
                            ownKnow = schedule.network[self.unique_id][app]['strength']
                        except KeyError:
                            edgesToAdd.append((self.unique_id, app, schedule.network[actorKey][app]['strength'] * self.model.commDensity))
                            continue
                        try:
                            otherKnow = schedule.network[actorKey][app]['strength']
                        except KeyError:
                            edgesToAdd.append((actorKey, app, ownKnow * self.model.commDensity))
                            continue
                        diff = ownKnow - otherKnow
                        if diff > 0:
                            schedule.network[actorKey][app]['strength'] += diff * self.model.commDensity
                        elif diff < 0:
                            schedule.network[self.unique_id][app]['strength'] += -diff * self.model.commDensity
                # as array: (app, node1, node2) aber dann keine Verbindung zu nodes
                
        schedule.network.add_weighted_edges_from(edgesToAdd, 'strength')

In [8]:
from utility import match
import time
import networkx as nx
import pandas as pd
import numpy as np

In [82]:
self = firstModel.schedule.actorList[0]
perceived=[0,1,0,1,0,1,0,1,0,1]
schedule=firstModel.schedule
time1 = 0.0
for i in range(1000):
    start1 = time.time()
    communicate1(self, schedule)
    stop1 = time.time()
    time1 += stop1 - start1

self = firstModel.schedule.actorList[0]
perceived=[0,1,0,1,0,1,0,1,0,1]
schedule=firstModel.schedule
time2 = 0.0
for i in range(1000):
    start2 = time.time()
    communicate2(self, schedule)
    stop2 = time.time()
    time2 += stop2 - start2

print(time1)
print(time2)

2.331467628479004
56.53080701828003


In [28]:
v1 = firstModel.schedule.appList[120].av
v2 = (0,1,0,1,0,1,0,1,0,1)

In [31]:
def sum1(v1, v2):
    sum = 0.0
    for a,b in zip(v1, v2):
        if a == '?':
            sum += 0.5
        elif int(a) == b:
            sum += 1.0
    return sum

def sum2(v1, v2):
    sum = 0.0
    for a,b in zip(v1, v2):
        try:
            if int(a) == b: sum += 1.0
        except ValueError:
            sum += 0.5
    return sum

In [32]:
print(sum1(v2, v2))

10.0


In [25]:
time1 = 0.0
for i in range(1000000):
    start1 = time.time()
    sum = sum1(v1, v2)
    stop1 = time.time()
    time1 += stop1 - start1

time2 = 0.0
for i in range(1000000):
    start2 = time.time()
    sum = sum2(v1, v2)
    stop2 = time.time()
    time2 += stop2 - start2

print(time1)
print(time2)

3.5280344486236572
9.025083780288696


In [11]:
import time
import numpy as np
from random import choices
from random import choice

In [44]:
schedule = firstModel.schedule
self = firstModel.schedule

In [1]:
from experiment import runExperiment
from multiprocessing import Pool
import csv

In [2]:
#10 runs in 271.2s
#for range in range(50,61):
#    print(runExperiment(range))

In [3]:
#10 runs in 109.5s
#1000 runs in 
results = set()
for i in range(2):
    p = Pool(8)
    results = results.union(p.map(runExperiment, [round(x * 0.01, 2) for x in range(101)]))
    p.close()
    p.join()
    print(f'Iteration {i + 1} finished.')

with open('output.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_NONNUMERIC)
    for r in results:
        writer.writerow(r)

Iteration 1 finished.
Iteration 2 finished.


In [5]:
with open('output.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_NONNUMERIC)
    for r in results:
        writer.writerow(r)

In [1]:
from experiment import runExperiment
from multiprocessing import Pool
from functools import partial

In [2]:
runExperiment(3, iterations=2)

Iteration 1 finished.
Iteration 2 finished.


{(14.92, 1710.835, 320443.85925757413, 220615.0, 34.66765170149808),
 (15.495, 1774.855, 337416.30955640046, 232829.0, 33.645709633106215),
 3}

In [4]:
p = Pool(8)
results = p.map(runExperiment, [(x) for x in range(5)]) #[round(x * 0.01, 2) for x in range(11)]
p.close()
p.join()

TypeError: 'int' object is not callable

In [4]:
[(x,10) for x in range(11)]

[(0, 10),
 (1, 10),
 (2, 10),
 (3, 10),
 (4, 10),
 (5, 10),
 (6, 10),
 (7, 10),
 (8, 10),
 (9, 10),
 (10, 10)]